For more examples on the Python API visit https://plot.ly/python/ <br>
Checkout the API reference for details at https://plot.ly/python/reference/ <br>
And other examples for MATLAB, R and JavaScript https://plot.ly/api/

Let's import Python's scientific computing library Numpy! To consider any time you use linear algebra.

In [1]:
import numpy as np
from IPython.display import display, Math, Latex

Let's import Plotly's libraries `plotly` and `graph_obj`.

In [2]:
import plotly.plotly as plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt

Then let's sign into our account! 
https://plot.ly/products/cloud/ <br>
Get your API key! 
https://plot.ly/settings/api <br>

In [3]:
plotly.sign_in('alexandres', 'nc66e5nb14')

Twin detectors of the Laser Interferometer Gravitational-Wave Observatory (LIGO) were placed in Livingston (30°N, 91°W) and Hanford (46°N 119°W). 

In [25]:
#############################################
# STEP1: DATA
#############################################
latitude = [30, 46]
longitude = [-91, -119]
names = ['Livingston', 'Hanford']

In [26]:
#############################################
# STEP2: TRACES
#############################################
trace_location = {
    'type': 'scattergeo',
    'lat': latitude,
    'lon': longitude,
    'text': names,
    'marker': {
        'size': 10,
        'color': ['rgb(255, 0, 0)','rgb(0,0,0)']
    }
}

In [27]:
#############################################
# STEP3: LAYOUT
#############################################
layout_location = {
    'title': 'Where are Hanford and Livingston LIGO locations?',
    'geo': {
        'projection': {
            'type': 'orthographic'
        }
    }       
}

In [29]:
#############################################
# STEP4: PLOT!
#############################################
data_location = go.Data([trace_location])
figure_location = go.Figure(data = data_location, layout = layout_location)
plotly.iplot(figure_location, filename = 'LIGO-locations')

Download data and include `readligo` library to read LIGO data from the LIGO website https://losc.ligo.org

In [11]:
import readligo as rl

Read data from the data file.

In [12]:
import json
try:
    events = json.load(open("O1_events.json","r"))
except IOError:
    print("Cannot find resource file O1_events.json; download it from https://losc.ligo.org/s/events/O1_events.json")

# Extract the parameters for the desired event:
event = events['GW151226']
fn_H1 = event['fn_H1']              # File name for H1 data
fn_L1 = event['fn_L1']              # File name for L1 data
fn_template = event['fn_template']  # File name for template waveform
fs = event['fs']                    # Set sampling rate
tevent = event['tevent']            # Set approximate event GPS time
fband = event['fband']              # frequency band for bandpassing signal

Let's make some Scatter plots! Use the Plotly cheatsheet to get going quickly! https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf

In [14]:
#############################################
# STEP1: DATA
#############################################
# First from H1
strain_H1, time_H1, chan_dict_H1 = rl.loaddata(fn_H1, 'H1')
# and then from L1
strain_L1, time_L1, chan_dict_L1 = rl.loaddata(fn_L1, 'L1')

# both H1 and L1 will have the same time vector, so:
time = time_H1
delta_t = 10
indxt = np.where((time >= tevent - delta_t) & (time < tevent + delta_t))
time_seg = time[indxt] - tevent

In [22]:
#############################################
# STEP2: TRACES
#############################################
trace_time_series_H1 = {
    'type': 'scatter',
    'name': 'H1',
    'x': time_seg,
    'y': strain_H1
}
trace_time_series_L1 = {
    'type': 'scatter',
    'name': 'L1',
    'x': time_seg,
    'y': strain_L1
}

In [19]:
#############################################
# STEP3: LAYOUT
#############################################
layout_time_series = {
    'showlegend': True,
    'title': "LIGO strain data " + str(delta_t) + ' around of the event',
    'xaxis': {
        'title': 'time [s] since ' + str(tevent) + ' sec',
        'titlefont': {
            'family': 'Arial, monospace',
            'size': 18,
            'color': '#ff6600'
        }
    },
    'yaxis': {
        'title': '',
        'titlefont': {
            'family': 'Arial, monospace',
            'size': 18,
            'color': '#ff6600'
        }
    }
}    

In [23]:
#############################################
# STEP4: PLOT AWAY
#############################################
data_time_series = go.Data([trace_time_series_H1, trace_time_series_L1])
fig_time_series = go.Figure(data = data_time_series, layout = layout_time_series)
plotly.iplot(fig_time_series, filename = 'LIGO-time-series')

The draw time for this plot will be slow for all clients.


Next, let's try plotting the data in the Fourier domain, using a Fast Fourier Transform (FFT). Because LIGO data has a great deal of excess noise at low frequencies, spectral leakage is often an issue. To mitigate this, we'll apply a Blackman Window before we take the FFT: 

In [17]:
formula_fft = 'X_k = \sum_{N=0}^{N-1} x_n  e^\dfrac{-2\pi k n}{N} \ , \ k \ \epsilon \ \mathbb{Z}'
display(Math(formula_fft))

<IPython.core.display.Math object>

In [18]:
t_tot = time[-1] - time[0]
dt = time[1] - time[0]
fs = int(1.0 / dt)

window = np.blackman(strain_H1.size)
windowed_strain_H1 = strain_H1*window
windowed_strain_L1 = strain_L1*window

In [19]:
#############################################
# DATA
#############################################
freq = np.arange(0, fs, 1.0/t_tot)

freq_magnitude_H1 = abs(np.fft.fft(windowed_strain_H1))/t_tot
freq_magnitude_L1 = abs(np.fft.fft(windowed_strain_L1))/t_tot

In [20]:
#############################################
# TRACES
#############################################
trace_fft_H1 = go.Scatter(
    name = 'H1 magnitude',
    x = freq,
    y = freq_magnitude_H1
)
trace_fft_L1 = go.Scatter(
    name = 'L1 magnitude',
    x = freq,
    y = freq_magnitude_L1
)

In [21]:
#############################################
# LAYOUT
#############################################
layout_fft = go.Layout(
    showlegend = True,
    title = 'Strain in Frequency Domain',
    xaxis = {
        'title': 'frequency [Hz]',
        'type': 'log',
        'titlefont': {
            'family': 'Arial, monospace',
            'size': 18,
            'color': '#ff6600'
        }
    },
    yaxis = {
        'title': '$ Hz^{1/2} $',
        'type': 'log',
        'titlefont': {
            'family': 'Arial, monospace',
            'size': 18,
            'color': '#ff6600'
        }
    }
)   

In [22]:
data_fft = go.Data([trace_fft_H1, trace_fft_L1])
fig_fft = go.Figure(data = data_fft, layout = layout_fft)

In [23]:
plotly.iplot(fig_fft, filename = 'LIGO-frequency-distribution')

The draw time for this plot will be slow for all clients.


In [ ]:
#############################################
# DATA
#############################################
fileName = 'H-H1_LOSC_4_V1-815411200-4096.hdf5'
strain, time, channel_dict = rl.loaddata(fileName)
ts = time[1] - time[0]      #-- Time between samples
fs = int(1.0 / ts)          #-- Sampling frequency

#-- Choose a few seconds of "good data"
segList = rl.dq_channel_to_seglist(channel_dict['DEFAULT'], fs)
length = 16  # seconds
strain_seg = strain[segList[0]][0:(length*fs)]
time_seg = time[segList[0]][0:(length*fs)]


NFFT = 1024
window = np.blackman(NFFT)
spec_power, freqs, bins, im = plt.specgram(strain_seg, NFFT=NFFT, Fs=fs, 
                                    window=window)
plt.xlabel('Time (s)')
plt.ylabel('Freq (Hz)')


In [ ]:
plt.show()

In [ ]:
#############################################
# TRACES
#############################################
trace_frequency_heatmap_H1 = go.Heatmap(
    z = spec_power,
    y = freqs,
    x = bins
)

In [ ]:
#############################################
# LAYOUT
#############################################


In [ ]:
data_frequency_heatmap = go.Data([trace_frequency_heatmap_H1])

In [ ]:
plotly.iplot([trace_frequency_heatmap_H1], filename = 'frequency_heatmap')

In [ ]:
import plotly.tools as tls

In [ ]:
tls.embed('https://plot.ly/~empet/13564/gravitational-waves/')

In [ ]:
tls.embed('https://plot.ly/~alexandres/115/ok-hs-physics-teachers/')

In [ ]:
tls.embed('https://plot.ly/~alexandres/114/')

In [ ]:
tls.embed('https://plot.ly/~empet/13810/')

In [ ]:
tls.embed('https://plot.ly/~empet/13870/')

In [ ]:
tls.embed('https://plot.ly/~empet/13872')